# Урок 5. Part-of-Speech разметка, NER, извлечение отношений

## Домашнее задание

### Задание 1. Написать теггер на данных с русским языком
1. проверить `UnigramTagger`, `BigramTagger`, `TrigramTagger` и их комбинации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы, сделать выводы

In [ ]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyconll

In [ ]:
#!git clone https://github.com/named-entity/nltk4russian.git
#!python /content/nltk4russian/setup.py install
#import nltk4russian
#from nltk4russian.tagger import PMContextTagger

In [ ]:
!mkdir datasets

In [ ]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2023-01-26 11:21:51--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-01-26 11:21:51 ERROR 404: Not Found.

--2023-01-26 11:21:51--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14704579 (14M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-dev.conllu’

./datasets/ru_synta 100%[===================

In [ ]:
full_train = pyconll.load_from_file('/content/datasets/ru_syntagrus-ud-dev.conllu')
full_test = pyconll.load_from_file('/content/datasets/ru_syntagrus-ud-dev.conllu')

In [ ]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Алгоритм NOUN
, PUNCT
от ADP
имени NOUN
учёного NOUN
аль PART
- PUNCT
Хорезми PROPN
, PUNCT
- PUNCT
точный ADJ
набор NOUN
инструкций NOUN
, PUNCT
описывающих VERB
порядок NOUN
действий NOUN
исполнителя NOUN
для ADP
достижения NOUN
результата NOUN
решения NOUN
задачи NOUN
за ADP
конечное ADJ
время NOUN
. PUNCT

В ADP
старой ADJ
трактовке NOUN
вместо ADP
слова NOUN
" PUNCT
порядок NOUN
" PUNCT
использовалось VERB
слово NOUN
" PUNCT
последовательность NOUN
" PUNCT
, PUNCT
но CCONJ
по ADP
мере NOUN
развития NOUN
параллельности NOUN
в ADP
работе NOUN
компьютеров NOUN
слово NOUN
" PUNCT
последовательность NOUN
" PUNCT
стали VERB
заменять VERB
более ADV
общим ADJ
словом NOUN
" PUNCT
порядок NOUN
" PUNCT
. PUNCT



In [ ]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [ ]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(str(token.form)) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 144
Наибольшая длина токена 30


In [ ]:
all_train_texts = [' '.join(str(token.form) for token in sent) for sent in full_train]
all_test_texts = [' '.join(str(token.form) for token in sent) for sent in full_test]

all_train_labels = [' '.join(str(token.upos) for token in sent) for sent in full_train]
all_test_labels = [' '.join(str(token.upos) for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Алгоритм , от имени учёного аль - Хорезми , - точный набор инструкций , описывающих порядок действий исполнителя для достижения результата решения задачи за конечное время .
В старой трактовке вместо слова " порядок " использовалось слово " последовательность " , но по мере развития параллельности в работе компьютеров слово " последовательность " стали заменять более общим словом " порядок " .
Это связано с тем , что работа каких-то инструкций алгоритма может быть зависима от других инструкций или результатов их работы .
Таким образом , некоторые инструкции должны выполняться строго после завершения работы инструкций , от которых они зависят .
Независимые инструкции или инструкции , ставшие независимыми из-за завершения работы инструкций , от которых они зависят , могут выполняться в произвольном порядке , параллельно или одновременно , если это позволяют используемые процессор и операционная система .
Ранее часто писали " алгорифм " , сейчас такое написание используется редко , но , т

In [ ]:
# from nltk.corpus import brown
#nltk.download('brown')
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger
#brown_tagged_sents = brown.tagged_sents(categories='news')
#train_data = brown_tagged_sents[:int(len(brown_tagged_sents) * 0.9)]
unigram_tagger = UnigramTagger(fdata_train)
bigram_tagger = BigramTagger(fdata_train)#, backoff=unigram_tagger) 
trigram_tagger = TrigramTagger(fdata_train)

In [ ]:
display(unigram_tagger.tag(fdata_sent_test[100]), unigram_tagger.evaluate(fdata_test))

<ipython-input-12-fd8c468215e5>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(unigram_tagger.tag(fdata_sent_test[100]), unigram_tagger.evaluate(fdata_test))


[('Это', 'PRON'),
 ('сочинение', 'NOUN'),
 ('известно', 'ADJ'),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', 'NOUN'),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', 'NOUN'),
 ('XVI', 'ADJ'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.970551468194544

In [ ]:
display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.evaluate(fdata_test))

<ipython-input-13-d56645bf4924>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(bigram_tagger.tag(fdata_sent_test[100]), bigram_tagger.evaluate(fdata_test))


[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', None),
 ('многих', None),
 ('вариантах', None),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', 'VERB'),
 ('к', 'ADP'),
 ('ещё', 'ADV'),
 ('более', 'ADV'),
 ('древним', 'ADJ'),
 ('рукописям', 'NOUN'),
 ('XVI', 'ADJ'),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.9604661761833453

In [ ]:
display(trigram_tagger.tag(fdata_sent_test[100]), trigram_tagger.evaluate(fdata_test))

<ipython-input-14-ed79a29e8596>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(trigram_tagger.tag(fdata_sent_test[100]), trigram_tagger.evaluate(fdata_test))


[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', None),
 ('многих', None),
 ('вариантах', None),
 ('(', None),
 ('самые', None),
 ('ранние', None),
 ('из', None),
 ('них', None),
 ('почти', None),
 ('на', 'ADP'),
 ('сто', None),
 ('лет', None),
 ('старше', None),
 (')', None),
 ('и', None),
 ('восходит', None),
 ('к', None),
 ('ещё', None),
 ('более', None),
 ('древним', None),
 ('рукописям', None),
 ('XVI', None),
 ('в', None),
 ('.', 'PUNCT')]

0.8668663324435184

Посмотрим на точность при комбинации теггеров

In [ ]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 

tag.evaluate(fdata_test) 


<ipython-input-15-9b1f8847150d>:12: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  tag.evaluate(fdata_test)


0.9828764893547757

При комбинации теггеров выходит чуть лучше

Создаем свой теггер

In [ ]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(str(tok[0]))
        train_label.append('NO_TAG' if str(tok[1]) is None else str(tok[1]))
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(str(tok[0]))
        test_label.append('NO_TAG' if str(tok[1]) is None else str(tok[1]))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [ ]:
test_enc_labels = le.transform(test_label)

In [ ]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM',
       'None', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB',
       'X'], dtype='<U5')

In [ ]:
hvectorizer = HashingVectorizer(ngram_range=(1, 2), analyzer='char', n_features=500)
hvectorizerw = HashingVectorizer(ngram_range=(1, 2), analyzer='word', n_features=500)

In [ ]:
X_train = hvectorizer.fit_transform(train_tok)
X_trainw = hvectorizerw.fit_transform(train_tok)

In [ ]:
X_test = hvectorizer.transform(test_tok)
X_testw = hvectorizerw.transform(test_tok)

In [ ]:
X_train.shape

(153590, 500)

In [ ]:
X_trainw.shape

(153590, 500)

In [ ]:
%%time
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)
lr.fit(X_trainw, train_enc_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CPU times: user 30.1 s, sys: 105 ms, total: 30.2 s
Wall time: 31 s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [ ]:
pred = lr.predict(X_test)
predw = lr.predict(X_testw)

In [ ]:
accuracy_score(test_enc_labels, pred)

0.20921283937756363

In [ ]:
accuracy_score(test_enc_labels, predw)

0.3402500162771014

#### Вывод:  
Если основываться только на данных методах, я считаю что `UnigramTagger` лучше, но взял бы `BigramTagger`, так как считаю, что данные из него более точнее

### Задание 2. Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/
1. проверить NER из nltk/spacy/deeppavlov.
2. написать свой NER, попробовать разные подходы:
    - передаём в сетку токен и его соседей.
    - передаём в сетку только токен.
    - свой вариант.
3. сравнить свои реализованные подходы на качество — вывести `precision/recall/f1_score`.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('ieer')

def url_to_string(url, headers):
    res = requests.get(url, headers=headers)
    html = res.text
    print(res)
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36",
        "Accept-Encoding": "gzip, deflate",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en"
    }
url = 'https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news'
document = url_to_string(url, headers)

nltk.pos_tag(nltk.word_tokenize(document))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package ieer to /root/nltk_data...
[nltk_data]   Unzipping corpora/ieer.zip.


<Response [200]>


[('F.B.I', 'NNP'),
 ('.', '.'),
 ('Agent', 'NNP'),
 ('Peter', 'NNP'),
 ('Strzok', 'NNP'),
 (',', ','),
 ('Who', 'NNP'),
 ('Criticized', 'NNP'),
 ('Trump', 'NNP'),
 ('in', 'IN'),
 ('Texts', 'NNP'),
 (',', ','),
 ('Is', 'NNP'),
 ('Fired', 'NNP'),
 ('-', ':'),
 ('The', 'DT'),
 ('New', 'NNP'),
 ('York', 'NNP'),
 ('Times', 'NNP'),
 ('SectionsSEARCHSkip', 'NNP'),
 ('to', 'TO'),
 ('contentSkip', 'VB'),
 ('to', 'TO'),
 ('site', 'NN'),
 ('indexPolitics', 'NNS'),
 ('Today', 'NNP'),
 ('’', 'NNP'),
 ('s', 'NN'),
 ('PaperPolitics|F.B.I', 'NNP'),
 ('.', '.'),
 ('Agent', 'NNP'),
 ('Peter', 'NNP'),
 ('Strzok', 'NNP'),
 (',', ','),
 ('Who', 'NNP'),
 ('Criticized', 'NNP'),
 ('Trump', 'NNP'),
 ('in', 'IN'),
 ('Texts', 'NNP'),
 (',', ','),
 ('Is', 'VBZ'),
 ('Firedhttps', 'NNP'),
 (':', ':'),
 ('//www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.htmlGive',
  'JJ'),
 ('this', 'DT'),
 ('articleGive', 'JJ'),
 ('this', 'DT'),
 ('articleGive', 'JJ'),
 ('this', 'DT'),
 ('article1625AdvertisementCont

In [ ]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Agent Peter Strzok', 'PERSON'),
 ('Andrew G. McCabe', 'PERSON'),
 ('Anthony D. Weiner', 'PERSON'),
 ('Army', 'ORGANIZATION'),
 ('China', 'GPE'),
 ('Christopher', 'PERSON'),
 ('Clinton', 'PERSON'),
 ('CompanyNYTCoContact', 'ORGANIZATION'),
 ('Congress', 'ORGANIZATION'),
 ('David Bowdich', 'PERSON'),
 ('Director Wray', 'PERSON'),
 ('F.B.I.', 'ORGANIZATION'),
 ('Georgetown University', 'ORGANIZATION'),
 ('Goelman', 'PERSON'),
 ('Hillary', 'ORGANIZATION'),
 ('Hillary Clinton', 'PERSON'),
 ('Horowitz', 'PERSON'),
 ('House', 'ORGANIZATION'),
 ('Hundreds', 'PERSON'),
 ('Is', 'PERSON'),
 ('Is Fired', 'PERSON'),
 ('James B. Comey', 'PERSON'),
 ('Lisa Page', 'PERSON'),
 ('March', 'GPE'),
 ('Michael E. Horowitz', 'PERSON'),
 ('Michael S. SchmidtAug', 'PERSON'),
 ('Midyear Exam', 'PERSON'),
 ('Mr.', 'PERSON'),
 ('Mr. Bowdich', 'PERSON'),
 ('Mr. Goelman', 'PERSON'),
 ('Mr. Horowitz', 'PERSON'),
 ('Mr. McCabe', 'PERSON'),
 ('Mr. Mueller', 'PERSON'),
 ('Mr. Strzok', 'PERSON'),
 ('Mr. Trump', 'PERS

In [ ]:
!python -m venv env 
#.\env\Scripts\activate.bat
!pip install deeppavlov
!python -m deeppavlov install squad_bert

!python -m deeppavlov install ner_ontonotes

The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.8-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: ['/content/env/bin/python3', '-Im', 'ensurepip', '--upgrade', '--default-pip']

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.7/441.7 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3

In [ ]:
import deeppavlov
from deeppavlov import configs, build_model
deeppavlov_ner = build_model(configs.ner.ner_rus_bert, download=True, install=True)

2023-01-26 11:25:09.755 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/ner/ner_rus_bert_torch_new.tar.gz to /root/.deeppavlov/models/ner_rus_bert_torch_new.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/v1/ner/ner_rus_bert_torch_new.tar.gz to /root/.deeppavlov/models/ner_rus_bert_torch_new.tar.gz
100%|██████████| 1.44G/1.44G [00:48<00:00, 29.9MB/s]
2023-01-26 11:25:58.700 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/models/ner_rus_bert_torch_new.tar.gz archive into /root/.deeppavlov/models/ner_rus_bert_torch
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/models/ner_rus_bert_torch_new.tar.gz archive into /root/.deeppavlov/models/ner_rus_bert_torch


Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

In [ ]:
# ner_ontonotes_bert_mult

In [ ]:
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

[[['Нью',
   '-',
   'Йорк',
   ',',
   'США',
   ',',
   '30',
   'апреля',
   '2020',
   ',',
   '01',
   ':',
   '01',
   '—',
   'REGNUM',
   'В',
   'администрации',
   'президента',
   'США',
   'Дональда',
   'Трампа',
   'планируют',
   'пройти',
   'все',
   'этапы',
   'создания',
   'вакцины',
   'от',
   'коронавируса',
   'в',
   'ускоренном',
   'темпе',
   'и',
   'выпустить',
   '100',
   'млн',
   'доз',
   'до',
   'конца',
   '2020',
   'года',
   ',',
   'передаёт',
   'агентство',
   'Bloomberg',
   'со',
   'ссылкой',
   'на',
   'осведомлённые',
   'источники']],
 [['B-LOC',
   'I-LOC',
   'I-LOC',
   'O',
   'B-LOC',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-ORG',
   'O',
   'O',
   'O',
   'B-LOC',
   'B-PER',
   'I-PER',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-ORG',
   'O',
   

In [ ]:
# doc = ' '.join(words_docs)

Spacy

In [38]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-py3-none-any.whl size=98072933 sha256=56e01b6225e5a9609102b383727ff67d66c506567a995d0ac5d7a14e27846016
  Stored in directory: /root/.cache/pip/wheels/31/1f/21/52bc46cea1e50c10a3b246d5d266455b9a0e77c380bb7ac928
Successfully built en-core-web-md


In [39]:
from spacy import displacy
!python -m spacy download en_core_web_md
## import en_core_web_md
import en_core_web_md
nlp = en_core_web_md.load()
doc = nlp("I think Barack Obama met founder of Facebook at occasion of a release of a new NLP algorithm.")

displacy.render(doc, style="dep", jupyter=True) # (1)
displacy.render(doc, style="ent", jupyter=True) # (2)

2023-01-26 11:46:56.206855: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-md
    Found existing installation: en-core-web-md 2.2.0
    Uninstalling en-core-web-md-2.2.0:
      Successfully uninstalled en-core-web-md-2.2.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [40]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 KB 7.4 MB/s eta 0:00:00


In [41]:
import corus

In [42]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2023-01-26 11:47:21--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  2.29MB/s    in 0.8s    

2023-01-26 11:47:23 (2.29 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [43]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [44]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='455',
    text='Новым премьер-министром Крыма станет глава МВД Украины\r\n\r\nПрезидент Украины Виктор Янукович согласовал назначение главы МВД страны Анатолия Могилева на должность премьер-министра Автономной республики Крым. Об этом сообщается на официальном сайте украинского президента.\r\n\r\nВ соответствии с конституцией Украины, президент согласовывает кандидатуру премьер-министра Крыма с Верховным Советом автономной республики, после чего кандидат назначается на должность парламентом Крыма. Как сообщает "Украинская правда", вопрос о назначении Могилева премьером будет рассмотрен на внеочередном заседании крымского парламента 8 ноября.\r\n\r\nПосле согласования кандидатуры Могилева Янукович подписал указ о его увольнении с поста главы МВД. Новым министром внутренних дел назначен Виталий Захарченко, ранее возглавлявший налоговую службу Украины.\r\n\r\nПост главы правительства Крыма оставался незанятым с 17 августа 2011 года, когда скоропостижно скончался предыду

In [45]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
from razdel import tokenize
import pandas as pd

In [47]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [48]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [49]:
doc = df_words.word.values.tolist()

In [50]:
df_words['tag'].value_counts()

OUT         219110
PER          21189
ORG          13644
LOC           4561
GEOPOLIT      4352
MEDIA         2482
Name: tag, dtype: int64

In [51]:
df_words.head(20)

,word,tag
0,Президент,OUT
1,Ирана,GEOPOLIT
2,М,PER
3,.,PER
4,Ахмадинежад,PER
5,назначил,OUT
6,и,OUT
7,.,OUT
8,о,OUT
9,.,OUT


In [52]:
df_words.shape

(265338, 2)

In [53]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [54]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [55]:
train_x.apply(len).max(axis=0)

32

In [56]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [57]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [58]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10 

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int'
    #ngrams=(1, 3),
)
    # output_sequence_length=seq_len) # по идее если выставить этот параметр None то как раз будет с соседями, а так урезается до 10 это один токен

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [59]:
len(vectorize_layer.get_vocabulary())

29937

In [60]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(1, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [61]:
mmodel = modelNER()

In [62]:
'''
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics='accuracy')
'''
mmodel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.Precision()])


In [63]:
mmodel.fit(train_data, validation_data=valid_data, epochs=1)

12438/12438 [==============================] - 211s 17ms/step - loss: 0.0000e+00 - precision: 0.9835 - val_loss: 0.0000e+00 - val_precision: 0.9840


In [64]:
mmodel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.Recall()])

In [65]:
mmodel.fit(train_data, validation_data=valid_data, epochs=1)

12438/12438 [==============================] - 203s 16ms/step - loss: 0.0000e+00 - recall: 1.0000 - val_loss: 0.0000e+00 - val_recall: 1.0000


In [73]:
!pip install --upgrade tensorflow 
!pip install --upgrade keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [78]:
mmodel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc',f1_m,precision_m, recall_m])

In [79]:
mmodel.fit(train_data, validation_data=valid_data, epochs=1)

12438/12438 [==============================] - 322s 26ms/step - loss: 0.0000e+00 - acc: 0.0175 - f1_m: 0.9914 - precision_m: 0.9835 - recall_m: 1.0000 - val_loss: 0.0000e+00 - val_acc: 0.0164 - val_f1_m: 0.9917 - val_precision_m: 0.9840 - val_recall_m: 1.0000
